In [ ]:
import functions_framework
import pandas as pd
from sqlalchemy import create_engine
import requests
import pytz
from datetime import datetime, timedelta
#Please create python file to save your sql password and user API Key
from safe_key import safe_key

@functions_framework.http
def update_flight_data(request):
  connection_string = connection()
  #Retrieving information from sql airports basic information
  airports_from_db = pd.read_sql("airports", connection_string)
  
  #Update weather data
  FlightData(airports_from_db, connection_string)
  return "Flight data successfully added"

def connection():
  schema = "gans_cloud"
  host = safe_key["Host_IP"]
  user = "root"
  port = 3306
  password = safe_key["SQL_cloud_password"]

  db = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
  return db

#Sending python data to sql table
def send_data(df, connection_string, sql_table):
  return df.to_sql(name = sql_table,
                  con = connection_string,
                  if_exists ='append',
                  index = False) #Ensures the DataFrame’s index is not written as a separate column in the database table.


def get_timezone_by_icao(icao_code):
  try:
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{icao_code}"
    
    headers = {
        "x-rapidapi-key": safe_key["x_rapidapi_key"],
        "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
      data = response.json()
      timezone_str = data["timeZone"]
      return timezone_str
    else:
      print(f"Failed to fetch timezone for ICAO: {icao_code}")
      return "UTC"
  except Exception as e:
    print(f"Error in timezone lookup for ICAO: {icao_code}: {e}")
    return "UTC"
    
def FlightData(airports_from_db, connection_string):
  flight_data = []

  for code_row in airports_from_db.itertuples(index=False):
    code = code_row.icao
    Airport_name = code_row.Airport_name
    City_id = code_row.City_id

    # Find the city timezone based on icao code API
    timezone_str = get_timezone_by_icao(code) or "UTC"
    city_timezone = pytz.timezone(timezone_str)

    # Get today and tomorrow's dates
    today = datetime.now(city_timezone).date()
    tomorrow = today + timedelta(days=1)

    # API requires two 12-hour calls
    times = [["00:00", "11:59"], ["12:00", "23:59"]]

    for time in times:

      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{code}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      
      querystring = {
                      "withLeg": "true",
                      "direction": "Arrival",
                      "withCancelled": "false",
                      "withCodeshared": "true",
                      "withCargo": "false",
                      "withPrivate": "false"
                  }
      
      headers = {
          "x-rapidapi-key": safe_key["x_rapidapi_key"],
          "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
          }

      # API request
      response = requests.get(url, headers=headers, params=querystring)
          
      if response.status_code != 200:
        print(f"Error fetching flight data for ICAO: {code}, Status Code: {response.status_code}")
        continue

      try:
        flight = response.json()
      except requests.exceptions.JSONDecodeError:
        print(f"Invalid JSON response for flight data for ICAO: {code}")
        continue

      # Data retrieval timestamp
      retrieval_time = datetime.now(city_timezone).strftime("%Y-%m-%d %H:%M:%S")

      # Parse arrivals
      for item in flight.get("arrivals", []):

        #This skip step if arrival_time is not available
        arrival_time = item["arrival"]["scheduledTime"].get("local", None)
        if not arrival_time:
            continue

        data = {
            "Arrival_airport_icao": code,
            "Arrival_airport_name": Airport_name,
            "Departure_airport_icao": item["departure"]["airport"].get("icao", None),
            "Flight_number": item.get("number", None),
            "Arrival_time": arrival_time,
            "Data_retrieved_time": retrieval_time,
            "City_id" : City_id
        }

        flight_data.append(data)

  #Create DataFrame
  flight_df = pd.DataFrame(flight_data)

  if not flight_df.empty:
    flight_df["Arrival_time"] = pd.to_datetime(flight_df["Arrival_time"].str[:-6])
    flight_df["Data_retrieved_time"] = pd.to_datetime(flight_df["Data_retrieved_time"])

  #To drop duplicate entries
  existing_flight_df = pd.read_sql("flights", con = connection_string)

  merged_df = flight_df.merge(existing_flight_df, on=["Flight_number", "Arrival_time"], how="left", indicator=True, suffixes= ("", "_existing"))

  #Its create merge column automatically bez of indicator=True
  flights_to_db = merged_df.loc[merged_df["_merge"] == "left_only", flight_df.columns]  
  
  send_data(flights_to_db, connection_string, "flights")

  return
 

In [11]:
from flask import Request
import json
request_data = {}
request = Request.from_values(data=json.dumps(request_data))

response = update_flight_data(request)
print(response)

Flight data successfully added
